In [1]:
%matplotlib inline
import glob
import os
import numpy as np
import numba as nb
import awkward as ak
import matplotlib.pyplot as plt
import uproot
from uproot_methods import TLorentzVectorArray
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
from cycler import cycler
import coffea.processor as processor

In [2]:
fileset = {'tHq': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/THQ_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/100000/38E83594-51BD-7D46-B96D-620DD60078A7.root',
                    'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/THQ_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/100000/3A3BA22C-AA71-2544-810A-6DF4C6BA96FC.root',
                    'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/THQ_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/100000/3AFB1F42-BC6D-D44E-86FD-DB93C83F88FF.root',
                    'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/THQ_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/100000/A37B4B7A-FB5B-484D-8577-40B860D77D23.root',
                    'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/THQ_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/100000/E3C7548E-EE40-BA45-9130-17DF56FBE537.root',
                    'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/THQ_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/100000/F9EFC559-09E9-BB48-8150-9AA8B7F02C1C.root'],
             'tHW': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/THW_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/70000/2806293E-D1DD-4A49-A274-0CC3BA57BBDF.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/THW_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/70000/2F19962E-1DFB-A14A-91C2-30B69D5651D3.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/THW_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/70000/D9744111-ED04-3F47-A52A-C18424F01609.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/THW_Hincl_13TeV-madgraph-pythia8_TuneCUETP8M1/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/70000/E4CFA095-E7DB-B449-986D-1A5D21FD1D50.root'],
             'ttW': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/30000/2E92C41D-6F94-F84B-A46B-61EDFFBEFE5B.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/30000/69D60020-75AF-CA42-8D94-99957EEF71DC.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/30000/AB62AEC8-6F15-1947-9E22-0059E196EBE5.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/30000/EA538168-AF02-774C-9648-9BF72471384D.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/2CF42572-162F-F14F-B7CA-96DABCD83589.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/4B8C7791-6DFF-5748-B6AF-7FE0F6B5CA1D.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/5BF4BF41-CFDB-F447-B034-D806DFE3252C.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/794E6532-E576-FB41-A23F-CD078AABC85E.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/8102B971-D215-004C-93EE-AE9A6C1907B3.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/EF9DC33E-137A-0640-976B-8C3AC89F534E.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/FF4BF1D3-A06A-0F44-8458-37984DEA5CAD.root'],
             'ttWJets': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttWJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/028BA117-69BE-B74D-9862-2170CA64B11C.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttWJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/1ABC137E-5233-044E-B3FE-8574DC6FC943.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttWJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/2274637A-76F8-4049-B651-F60817E6F775.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttWJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/2E4A4356-C68F-1B4C-A920-656C7F519328.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttWJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/3BEEC74F-9E73-1745-BCA2-09E2725C3979.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttWJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/80CEEF63-623A-8144-AEE6-0BAEB5BC314B.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttWJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/DA330C53-B0BE-A84F-9308-1BBBC38B095C.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttWJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/E3EE47DE-4DBB-FC48-84AB-4F9C26B9CCE8.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttWJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/F597371F-E866-6B47-A185-268F927E3372.root'],
            'ttZ': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/40000/5B1499CC-2B27-4F44-95BD-7B1C9907ACCD.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/40000/CDA99173-3C05-1A47-AE70-587B44E6348B.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/60000/28C6248C-D55F-8044-BDFB-696BAC0FE329.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/60000/F8A11DF6-4013-6F4C-BBE9-1759933FF20F.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/110000/308939D7-DA54-E949-958D-7B495723DB4C.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/110000/3A1F9F01-E4BC-A347-8D57-8DBFAA890EAB.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/110000/BF1A0533-7A59-A04C-9A6C-6A5EA4697A06.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/191705EA-1891-6942-8CD2-CD70B0E58893.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/27BD2647-8A2F-7C45-964F-F0277A0A011D.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/56608BD3-2A0E-1941-8386-F498EEC16A61.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/7D898E0C-0752-B948-8BA7-7016BD151BD2.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/97637969-BD4B-0640-8158-17722CB9F631.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/9BEAB5BD-5C7E-B54A-B42E-D81FA4B30A2B.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/C7073B6B-C69E-1B42-B378-7FAC8FF4E856.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext2-v1/120000/FF986F85-71C4-064D-9759-B0C1ACAE9454.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext3-v1/130000/34275A2F-313A-6D4D-A7EC-DD18BF9C413B.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext3-v1/130000/392B4635-E8FF-924C-A5B5-51D19C670F69.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext3-v1/130000/4C139140-A677-AB49-9DF9-5BBF4FA850D2.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext3-v1/130000/80416EBE-074F-5C4A-AE5F-6403A9F9E9D9.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext3-v1/130000/96F276D6-1C38-9F4F-93CE-503B860AB550.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext3-v1/130000/A1AD2099-D5D6-7A47-AEB6-562F68B04441.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext3-v1/130000/AAD8B507-CEC2-CE44-9B23-822A451F426C.root'],
            'ttZJets': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/12857BB7-7239-B94A-A890-7CB3C79BD057.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/193EA450-873A-3047-BA69-3B1D1EABB771.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/21167A9A-2FDD-C240-AF27-FAA935B0057C.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/576C9504-E9B2-484C-88DB-1F087D4BA6A3.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/5C781945-9A98-7F44-B397-F806603C9A01.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/69E6121A-F0DC-8043-991A-049199E4BCC4.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/72EF6B33-D0AA-6540-BA4A-A760E8194589.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/8B2D64F4-D873-124B-AE6A-0503FF7A0427.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/B865F28C-D9B3-0944-8E01-D0D3B01C4F26.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/C2C66219-4FAA-4C4E-95FB-CA259749AAA4.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/DD8A6F4F-52CF-9142-B264-B402A94CFD09.root',
                         'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttZJets_13TeV_madgraphMLM/NANOAODSIM/Nano1June2019_102X_mcRun2_asymptotic_v7-v1/40000/E0D9AB84-E0BF-0C49-9B2E-E003587FF335.root'],
            'ttH': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/05371F7D-77DA-FD4E-BF1F-9C4B6C7B2BF4.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/1097F45B-C2CC-034A-98D5-6A2C38F9E957.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/144E050B-AC6C-5046-B9BF-3CB1045651D1.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/1F8A2052-DD4D-F24B-AEF7-C6C8DB5E42AC.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/2BDDFDC0-1D0E-4141-BCB6-98FDE981E522.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/3D3743BD-A2B7-F34E-BBD3-FE98A055B935.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/70319D0B-46D5-6840-9FA3-49B990C6E117.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/7CF51AAB-6C37-F84E-B091-48C8217EE65E.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/90463B9A-2AB6-1D4E-9D7F-4E6AA6B4500C.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/A922DC1E-905C-0240-81D7-8470159370D4.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/C6BD7A39-4592-F84F-99FB-52C1BBC3F066.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/EE7FEAE7-21B8-4547-A0C7-589B3DDA0704.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/100000/F5A64CA8-059C-4D46-A2C4-9B679A4B911F.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ttHJetToNonbb_M125_13TeV_amcatnloFXFX_madspin_pythia8_mWCutfix/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/120000/83CB02F7-FFA9-CD47-9EE6-4D444843595E.root'],
            'WZ': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/120000/222071C0-CF04-1E4B-B65E-49D18B91DE8B.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/120000/7EC09E9F-A061-6D46-83DB-E91A8479CD7A.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/120000/9FA39757-C2AB-F34E-A2E8-3D728B605FD1.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/120000/E539344F-0815-064E-BFB2-14D265A460FA.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/0571F847-7582-0D43-A65D-BB1DF150BE77.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/1875117C-8380-0C45-A701-B221443E9010.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/23490416-1F0E-364F-B5D9-7F7C92360277.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/423A6069-CE53-A94F-8D9F-75342F9F9273.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/42816FAA-2580-7146-93BE-C9482FC56E5F.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/4DCA8EC5-BC5C-3447-8A91-C21142F57775.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/4F6290B2-15A7-C840-8544-26FBA7D62129.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/62527AE1-E838-BF44-AA33-C145C74F2196.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/64F0D9CB-C675-3F49-ACE2-E06CD5CFFA13.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/669C5D12-E263-0E41-9F49-B277198FA882.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/788641E9-D641-0F40-AC0D-69E335F7E04D.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/A2DCB4B2-9A3B-6049-B845-8BC13C623736.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/B5189B28-6A9D-9240-A943-830C76F1C4CE.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/CFBDFA7E-FA6A-9E40-976A-5E2CAE843CE2.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/D52DFC49-0016-2941-BEB6-7958728B782E.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/DD34D24C-D3E4-6046-AA01-EDF356AC83CB.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/E4035DC3-ACD2-0440-805F-F8C46406E0A0.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7_ext1-v1/110000/FED349E0-72BB-7E4F-B776-F9BB8C278B27.root'],
            'ZZ': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ZZTo4L_13TeV_powheg_pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/30000/010B000A-0883-064B-B250-A6159DB2603A.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ZZTo4L_13TeV_powheg_pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/30000/1B4E5034-DD6A-DB40-882D-20B5C42A9722.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ZZTo4L_13TeV_powheg_pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/30000/462F4643-6F37-EA42-8F6C-90B91C23A731.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ZZTo4L_13TeV_powheg_pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/30000/465A6EA9-E768-E642-8A73-B27E21BD3595.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ZZTo4L_13TeV_powheg_pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/30000/46E99F08-CB4C-914F-9CF0-1A9E7DCE8081.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ZZTo4L_13TeV_powheg_pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/30000/73D0D73F-E72D-E649-8772-2BECC3D296D7.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ZZTo4L_13TeV_powheg_pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/30000/773BDDB0-D100-DA4B-8522-17EB71265BEE.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ZZTo4L_13TeV_powheg_pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/30000/D08D925B-BA30-3642-9995-F165FC2542D7.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ZZTo4L_13TeV_powheg_pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/30000/DC582D74-81EF-504C-8342-08D42BB17C38.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ZZTo4L_13TeV_powheg_pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/30000/FD274208-7DF0-3245-945F-87B17415142E.root'],
            'ZZZ': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/ZZZ_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/37FA68CC-B841-7D41-994C-645CFA4BA227.root'],
            'WWZ': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WWZ_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/70000/8034D852-4E18-F441-B1B7-3A4BA82B8963.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WWZ_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/70000/EE791322-C349-A14A-B49A-75BDAA03482E.root'],
            'WZZ': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZZ_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/100000/4359B067-2773-4D48-A235-CD33C3A20AD9.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZZ_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/100000/50D0F171-4A01-644A-A1D9-6BF1D915DF2A.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WZZ_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/100000/61EDBB45-0A24-1847-9FD6-222D7D8D6AF3.root'],
            'tttt': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTTT_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/E23424C4-B553-114D-A9F2-9E2F21D641F6.root',
                     'root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/TTTT_TuneCUETP8M1_13TeV-amcatnlo-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/250000/ED3B459F-A945-C847-9B80-B847C3487049.root'],
            'WpWp': ['root://xrootd.unl.edu//store/mc/RunIISummer16NanoAODv5/WpWpJJ_EWK-QCD_TuneCUETP8M1_13TeV-madgraph-pythia8/NANOAODSIM/PUMoriond17_Nano1June2019_102X_mcRun2_asymptotic_v7-v1/120000/025F6345-BE37-DF45-B582-25EA8300B4E8.root'],
          }

In [4]:
class SignalProcessor(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "")
        lll_3rdLepton_pT_axis = hist.Bin("lll_3rdLepton_pT", "3rd Lepton $p_T$ [GeV]", 18, 0, 90)
        lll_fwdJet_leadbJet_axis = hist.Bin("lll_fwdJet_leadbJet", "$\Delta \eta$ max fwd jet and leading bjet", 18, 0, 6)
        lll_fwdJet_subleadbJet_axis = hist.Bin("lll_fwdJet_subleadbJet", "$\Delta \eta$ max fwd jet and sub-leading bjet", 18, 0, 6)
        lll_fwdJet_closestLepton_axis = hist.Bin("lll_fwdJet_closestLepton", "$\Delta \eta$ max fwd jet and closest lepton", 18, 0, 6)
        lll_highpT_sslp_axis = hist.Bin("lll_highpT_sslp", "$\Delta \Phi$ highest $p_T$ same-sign lepton pair", 18, 0, 3.5)
        lll_jetMaxEta_axis = hist.Bin("lll_jetMaxEta", "max jet $|\eta|$ ($p_T$ > 25)", 18, 0, 5)
        lll_minDelR_axis = hist.Bin("lll_minDelR", "Minimum $\Delta$ R between any two leptons", 18, 0, 4)
        lll_nJet25_axis = hist.Bin("lll_nJet25", "N(jet, $p_T$ > 25, $|\eta|$ < 2.4)", 6, 2, 8)
        lll_nJetEta1_axis = hist.Bin("lll_nJetEta1", "N(jet, $|\eta|$ > 1) non CSV medium", 7, 1, 8)        
        lll_TotalCharge_axis = hist.Bin("lll_TotalCharge", "Total Lepton Charge", 5, -2, 3)
        
        mumu_subleadLepton_pT_axis = hist.Bin("mumu_subleadLepton_pT", "Sub-leading Lepton $p_T$ [GeV]", 18, 0, 144)
        mumu_fwdJet_leadbJet_axis = hist.Bin("mumu_fwdJet_leadbJet", "$\Delta \eta$ max fwd jet and leading bjet", 18, 0, 6)
        mumu_fwdJet_subleadbJet_axis = hist.Bin("mumu_fwdJet_subleadbJet", "$\Delta \eta$ max fwd jet and sub-leading bjet", 18, 0, 6)
        mumu_fwdJet_closestLepton_axis = hist.Bin("mumu_fwdJet_closestLepton", "$\Delta \eta$ max fwd jet and closest lepton", 18, 0, 6)
        mumu_highpT_sslp_axis = hist.Bin("mumu_highpT_sslp", "$\Delta \Phi$ highest $p_T$ same-sign lepton pair", 18, 0, 3.5)
        mumu_jetMaxEta_axis = hist.Bin("mumu_jetMaxEta", "max jet $|\eta|$ ($p_T$ > 40)", 18, 0, 5)
        mumu_minDelR_axis = hist.Bin("mumu_minDelR", "Minimum $\Delta$ R between any two leptons", 18, 0, 4)
        mumu_nJet25_axis = hist.Bin("mumu_nJet25", "N(jet, $p_T$ > 25, $|\eta|$ < 2.4)", 6, 2, 8)
        mumu_nJetEta1_axis = hist.Bin("mumu_nJetEta1", "N(jet, $|\eta|$ > 1) non CSV medium", 7, 1, 8)        
        mumu_TotalCharge_axis = hist.Bin("mumu_TotalCharge", "Total Lepton Charge", 5, -2, 3)
        
        emu_subleadLepton_pT_axis = hist.Bin("emu_subleadLepton_pT", "Sub-leading Lepton $p_T$ [GeV]", 18, 0, 144)
        emu_fwdJet_leadbJet_axis = hist.Bin("emu_fwdJet_leadbJet", "$\Delta \eta$ max fwd jet and leading bjet", 18, 0, 6)
        emu_fwdJet_subleadbJet_axis = hist.Bin("emu_fwdJet_subleadbJet", "$\Delta \eta$ max fwd jet and sub-leading bjet", 18, 0, 6)
        emu_fwdJet_closestLepton_axis = hist.Bin("emu_fwdJet_closestLepton", "$\Delta \eta$ max fwd jet and closest lepton", 18, 0, 6)
        emu_highpT_sslp_axis = hist.Bin("emu_highpT_sslp", "$\Delta \Phi$ highest $p_T$ same-sign lepton pair", 18, 0, 3.5)
        emu_jetMaxEta_axis = hist.Bin("emu_jetMaxEta", "max jet $|\eta|$ ($p_T$ > 40)", 18, 0, 5)
        emu_minDelR_axis = hist.Bin("emu_minDelR", "Minimum $\Delta$ R between any two leptons", 18, 0, 4)
        emu_nJet25_axis = hist.Bin("emu_nJet25", "N(jet, $p_T$ > 25, $|\eta|$ < 2.4)", 6, 2, 8)
        emu_nJetEta1_axis = hist.Bin("emu_nJetEta1", "N(jet, $|\eta|$ > 1) non CSV medium", 7, 1, 8)        
        emu_TotalCharge_axis = hist.Bin("emu_TotalCharge", "Total Lepton Charge", 5, -2, 3)
        
        ee_subleadLepton_pT_axis = hist.Bin("ee_subleadLepton_pT", "Sub-leading Lepton $p_T$ [GeV]", 18, 0, 144)
        ee_fwdJet_leadbJet_axis = hist.Bin("ee_fwdJet_leadbJet", "$\Delta \eta$ max fwd jet and leading bjet", 18, 0, 6)
        ee_fwdJet_subleadbJet_axis = hist.Bin("ee_fwdJet_subleadbJet", "$\Delta \eta$ max fwd jet and sub-leading bjet", 18, 0, 6)
        ee_fwdJet_closestLepton_axis = hist.Bin("ee_fwdJet_closestLepton", "$\Delta \eta$ max fwd jet and closest lepton", 18, 0, 6)
        ee_highpT_sslp_axis = hist.Bin("ee_highpT_sslp", "$\Delta \Phi$ highest $p_T$ same-sign lepton pair", 18, 0, 3.5)
        ee_jetMaxEta_axis = hist.Bin("ee_jetMaxEta", "max jet $|\eta|$ ($p_T$ > 40)", 18, 0, 5)
        ee_minDelR_axis = hist.Bin("ee_minDelR", "Minimum $\Delta$ R between any two leptons", 18, 0, 4)
        ee_nJet25_axis = hist.Bin("ee_nJet25", "N(jet, $p_T$ > 25, $|\eta|$ < 2.4)", 6, 2, 8)
        ee_nJetEta1_axis = hist.Bin("ee_nJetEta1", "N(jet, $|\eta|$ > 1) non CSV medium", 7, 1, 8)        
        ee_TotalCharge_axis = hist.Bin("ee_TotalCharge", "Total Lepton Charge", 5, -2, 3)
        
        self._accumulator = processor.dict_accumulator({
            'lll_3rdLepton_pT': hist.Hist("Events", dataset_axis, lll_3rdLepton_pT_axis),
            'lll_fwdJet_leadbJet': hist.Hist("Events", dataset_axis, lll_fwdJet_leadbJet_axis),
            'lll_fwdJet_subleadbJet': hist.Hist("Events", dataset_axis, lll_fwdJet_subleadbJet_axis),
            'lll_fwdJet_closestLepton': hist.Hist("Events", dataset_axis, lll_fwdJet_closestLepton_axis),
            'lll_highpT_sslp': hist.Hist("Events", dataset_axis, lll_highpT_sslp_axis),
            'lll_jetMaxEta': hist.Hist("Events", dataset_axis, lll_jetMaxEta_axis),
            'lll_minDelR': hist.Hist("Events", dataset_axis, lll_minDelR_axis),
            'lll_nJet25': hist.Hist("Events", dataset_axis, lll_nJet25_axis),
            'lll_nJetEta1': hist.Hist("Events", dataset_axis, lll_nJetEta1_axis),
            'lll_TotalCharge': hist.Hist("Events", dataset_axis, lll_TotalCharge_axis),
            'mumu_subleadLepton_pT': hist.Hist("Events", dataset_axis, mumu_subleadLepton_pT_axis),
            'mumu_fwdJet_leadbJet': hist.Hist("Events", dataset_axis, mumu_fwdJet_leadbJet_axis),
            'mumu_fwdJet_subleadbJet': hist.Hist("Events", dataset_axis, mumu_fwdJet_subleadbJet_axis),
            'mumu_fwdJet_closestLepton': hist.Hist("Events", dataset_axis, mumu_fwdJet_closestLepton_axis),
            'mumu_highpT_sslp': hist.Hist("Events", dataset_axis, mumu_highpT_sslp_axis),
            'mumu_jetMaxEta': hist.Hist("Events", dataset_axis, mumu_jetMaxEta_axis),
            'mumu_minDelR': hist.Hist("Events", dataset_axis, mumu_minDelR_axis),
            'mumu_nJet25': hist.Hist("Events", dataset_axis, mumu_nJet25_axis),
            'mumu_nJetEta1': hist.Hist("Events", dataset_axis, mumu_nJetEta1_axis),
            'mumu_TotalCharge': hist.Hist("Events", dataset_axis, mumu_TotalCharge_axis),
            'emu_subleadLepton_pT': hist.Hist("Events", dataset_axis, emu_subleadLepton_pT_axis),
            'emu_fwdJet_leadbJet': hist.Hist("Events", dataset_axis, emu_fwdJet_leadbJet_axis),
            'emu_fwdJet_subleadbJet': hist.Hist("Events", dataset_axis, emu_fwdJet_subleadbJet_axis),
            'emu_fwdJet_closestLepton': hist.Hist("Events", dataset_axis, emu_fwdJet_closestLepton_axis),
            'emu_highpT_sslp': hist.Hist("Events", dataset_axis, emu_highpT_sslp_axis),
            'emu_jetMaxEta': hist.Hist("Events", dataset_axis, emu_jetMaxEta_axis),
            'emu_minDelR': hist.Hist("Events", dataset_axis, emu_minDelR_axis),
            'emu_nJet25': hist.Hist("Events", dataset_axis, emu_nJet25_axis),
            'emu_nJetEta1': hist.Hist("Events", dataset_axis, emu_nJetEta1_axis),
            'emu_TotalCharge': hist.Hist("Events", dataset_axis, emu_TotalCharge_axis),
            'ee_subleadLepton_pT': hist.Hist("Events", dataset_axis, ee_subleadLepton_pT_axis),
            'ee_fwdJet_leadbJet': hist.Hist("Events", dataset_axis, ee_fwdJet_leadbJet_axis),
            'ee_fwdJet_subleadbJet': hist.Hist("Events", dataset_axis, ee_fwdJet_subleadbJet_axis),
            'ee_fwdJet_closestLepton': hist.Hist("Events", dataset_axis, ee_fwdJet_closestLepton_axis),
            'ee_highpT_sslp': hist.Hist("Events", dataset_axis, ee_highpT_sslp_axis),
            'ee_jetMaxEta': hist.Hist("Events", dataset_axis, ee_jetMaxEta_axis),
            'ee_minDelR': hist.Hist("Events", dataset_axis, ee_minDelR_axis),
            'ee_nJet25': hist.Hist("Events", dataset_axis, ee_nJet25_axis),
            'ee_nJetEta1': hist.Hist("Events", dataset_axis, ee_nJetEta1_axis),
            'ee_TotalCharge': hist.Hist("Events", dataset_axis, ee_TotalCharge_axis),
            'cutflow': processor.defaultdict_accumulator(int)
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, events):
        output = self.accumulator.identity()
        
        # btag working point numbers, for easy reference.
        loose_wp = 0.5425
        medium_wp = 0.8484

        # Initialize relevant event columns.
        dataset = events.metadata["dataset"]
        jets = events.Jet
        muons = events.Muon
        electrons = events.Electron

        # This is bookkeeping that tracks how many total events there are.
        output['cutflow']['all events'] += ak.size(events, axis=0)
        
        
        
        # # # # # # # # # # #
        # LEPTON SELECTIONS #
        # # # # # # # # # # #
        
        # Here we establish the loose muons array, based on provided cuts.
        loose_muons = muons[(np.abs(muons.eta) < 2.4) &
                      (np.abs(muons.dxy) < 0.05) & 
                      (np.abs(muons.dz) < 0.1) & 
                      (muons.sip3d < 8) &
                      (muons.miniPFRelIso_all < 0.4) &
                      (muons.looseId) &
                      (muons.pt > 5)]
        
        # Tight is loose with additional cuts; build tight array and add cuts.
        tight_muons = loose_muons[(loose_muons.pt > 15) &
                                  (loose_muons.tightCharge > 0) &
                                  (loose_muons.mediumId) &
                                  (loose_muons.mvaTTH > 0.9)]
        
        # This ends up getting referenced a lot. Let's not generate it every time!
        abs_eta = np.abs(electrons.eta)
        
        # Here we establish the loose electrons array, based on provided cuts.
        loose_electrons = electrons[(abs_eta < 2.5) & 
                              (np.abs(electrons.dxy) < 0.05) & 
                              (np.abs(electrons.dz) < 0.1) & 
                              (electrons.lostHits < 2) &
                              (electrons.miniPFRelIso_all < 0.4) &
                              (electrons.pt > 7) &
                              (((electrons.mvaFall17V2noIso > 0) & (abs_eta < 1.479)) | ((electrons.mvaFall17V2noIso > 0.7) & (abs_eta > 1.479) & (abs_eta < 2.5)))]
            
        # Now we build tight from loose, adding in tight's extra cuts.
        # Redefine abs_eta since we're now using loose.
        abs_eta = np.abs(loose_electrons.eta)
        
        tight_electrons = loose_electrons[(loose_electrons.pt > 15) &
                                         (loose_electrons.lostHits == 0) &
                                         (loose_electrons.tightCharge > 0) &
                                         (loose_electrons.convVeto) &
                                         (loose_electrons.mvaTTH > 0.90) &
                                         # Two ranges for sigma_ieie.
                                         (((abs_eta < 1.479) & (loose_electrons.sieie < 0.011)) | 
                                         ((abs_eta < 2.5) & (abs_eta > 1.479) & (loose_electrons.sieie < 0.03))) &
                                         # Two ranges for H/E
                                         (((abs_eta < 1.479) & (loose_electrons.hoe < 0.1)) | 
                                         ((abs_eta < 2.5) & (abs_eta > 1.479) & (loose_electrons.hoe < 0.07))) &
                                         # Two ranges for 1/E - 1/p
                                         (((abs_eta < 1.479) & (loose_electrons.eInvMinusPInv < 0.01) & (loose_electrons.eInvMinusPInv > -0.05)) |
                                         ((abs_eta < 2.5) & (abs_eta > 1.479) & (loose_electrons.eInvMinusPInv < 0.005) & (loose_electrons.eInvMinusPInv > -0.05)))]        
        
        # Don't want any jets below 25 pT.
        jets = jets[(jets.pt > 25)]
        
        # Add flavor to keep track of muons/electrons in the event selection.
        loose_muons = ak.with_field(loose_muons, 0, 'flavor')
        loose_electrons = ak.with_field(loose_electrons, 1, 'flavor')
        tight_muons = ak.with_field(tight_muons, 0, 'flavor')
        tight_electrons = ak.with_field(tight_electrons, 1, 'flavor')
        
        # # # # # # # # # #
        # EVENT SELECTION #
        # # # # # # # # # #
        
        # Jet masks can be generated simply, since we're staying in one collection.
        jetMask_medium = ak.sum((np.abs(jets.eta) < 2.4) & (jets.btagCSVV2 > medium_wp), axis=1) >= 1
        jetMask_loose = ak.sum(((np.abs(jets.eta) < 2.4) | ((jets.pt > 40) & (np.abs(jets.eta) > 2.4))) & (jets.btagCSVV2 <= loose_wp), axis=1) >= 1
        
        # Loose pre-selection. See definition of stack_leptons above to see how we construct arrays across leptons.
        tight_leptons = ak.with_name(ak.concatenate([tight_muons, tight_electrons], axis=1), 'PtEtaPhiMCandidate')
        loose_leptons = ak.with_name(ak.concatenate([loose_muons, loose_electrons], axis=1), 'PtEtaPhiMCandidate')
        loose_pairs = ak.argcombinations(loose_leptons, 2, fields=['i0', 'i1'])
        loose_pairs_mass = (loose_leptons[loose_pairs['i0']] + loose_leptons[loose_pairs['i1']]).mass
        looseMask = (ak.all(loose_pairs_mass >= 12, axis=1)) & (ak.num(loose_pairs_mass) > 0)
        
        # Dilepton selection.
        tight_pairs = ak.argcombinations(tight_leptons, 2, fields=['i0', 'i1'])
        # Set a flag to check if the dilepton arrays are false - skip event mask if so.
        di_empty = False
        if ak.sum(ak.num(tight_pairs)) == 0:
            di_empty = True
        
        if di_empty == False:
            # Add flavors to see what kind of pair we have. mu = 0, e = 1, so: mumu = 0, emu = 1, ee = 2.
            tight_types = (tight_leptons[tight_pairs['i0']].flavor + tight_leptons[tight_pairs['i1']].flavor)

            # Respectively; each event should have exactly 1 pair (2 leptons) with same charge
            #               one should have pT > 25 (pT > 15 is guaranteed by lepton selection)
            #               for ee events, check that mass from Z is < 10 to veto possible Z bosons.
            dileptonMask = ((ak.num(tight_pairs) == 1) & ak.any(tight_leptons[tight_pairs['i0']].charge == tight_leptons[tight_pairs['i1']].charge, axis=1)) & ak.any((tight_leptons[tight_pairs['i0']].pt > 25) | (tight_leptons[tight_pairs['i1']].pt > 25), axis=1) & ak.any((tight_types < 2) | (np.abs((tight_leptons[tight_pairs['i0']] + tight_leptons[tight_pairs['i1']]).mass - 91.18) > 10), axis=1)

            dilepton_eventMask = (looseMask) & (dileptonMask) & (jetMask_medium) & (jetMask_loose)

            # Final dilepton channels
            tight_ll = tight_leptons[dilepton_eventMask]
            tights_2mu = tight_ll[ak.sum(tight_ll.flavor, axis=1) == 0]
            tights_2e = tight_ll[ak.sum(tight_ll.flavor, axis=1) == 2]
            tights_emu = tight_ll[ak.sum(tight_ll.flavor, axis=1) == 1]
            jets_2mu = jets[dilepton_eventMask & (ak.num(tight_muons) == 2)]
            jets_2e = jets[dilepton_eventMask & (ak.num(tight_electrons) == 2)]
            jets_emu = jets[dilepton_eventMask & (ak.num(tight_muons) == 1) & (ak.num(tight_electrons) == 1)]
        
        # Trilepton selection.
        tight_triples = ak.argcombinations(tight_leptons, 3, fields=['i0', 'i1', 'i2'])
        # Set a flag to check if the trilepton arrays are false - skip event mask if so.
        tri_empty = False
        if ak.sum(ak.num(tight_triples)) == 0:
            tri_empty = True
        if tri_empty == False:
            tight_muon_pairs = ak.combinations(tight_muons, 2, fields=['i0', 'i1'])
            tight_electron_pairs = ak.combinations(tight_electrons, 2, fields=['i0', 'i1'])

            # Helper function which takes a pair, tells you if it is an OSSF with |m_ll - m_z| < 15. Reject these events.
            def ossf_zmass_check(dileptons):
                return ak.any((dileptons.i0.charge != dileptons.i1.charge) & 
                        (np.abs(91.18 - (dileptons.i0 + dileptons.i1).mass) < 15.), axis=1)
            
            # Respectively; we want events with exactly three tight leptons (one triple),
            #               one should have pT > 25, discard if OSSF dileptons are close to the Z mass.
            trileptonMask = (ak.num(tight_triples) == 1) & ak.any((tight_leptons[tight_triples['i0']].pt > 25) | (tight_leptons[tight_triples['i1']].pt > 25) | (tight_leptons[tight_triples['i2']].pt > 25), axis=1) & (~ossf_zmass_check(tight_muon_pairs)) & (~ossf_zmass_check(tight_electron_pairs))

            # May as well combine all of our masks into one (well, two) big one(s) for easier referencing.
            # Split dileptons and trileptons into separate channels, because it's easier to work with later.

            trilepton_eventMask = (looseMask) & (trileptonMask) & (jetMask_medium) & (jetMask_loose)

            # Final trilepton channel
            tight_lll = tight_leptons[trilepton_eventMask]
            jets_3lep = jets[trilepton_eventMask]
        
        
        # # # # # #
        # OUTPUTS #
        # # # # # #
        
        # Selecting the data that we want.
        if (tri_empty == False):
        # Three-lepton channel
            if (ak.sum(ak.num(tight_lll)) > 0):
                lll_3rdLepton_pT = ak.min(tight_lll.pt, axis=1)
                lll_lightjets = jets_3lep[jets_3lep.btagCSVV2 <= loose_wp]
                lll_bjets = jets_3lep[jets_3lep.btagCSVV2 > medium_wp]
                lll_fwd_lightjets = lll_lightjets[ak.local_index(lll_lightjets, axis=1) == ak.argmax(lll_lightjets.eta, axis=1)]
                lll_max_bjets = lll_bjets[ak.local_index(lll_bjets, axis=1) == ak.argmax(lll_bjets.pt, axis=1)]
                lll_nomax_bjets = lll_bjets[ak.local_index(lll_bjets, axis=1) != ak.argmax(lll_bjets.pt, axis=1)]
                lll_submax_bjets = lll_nomax_bjets[ak.local_index(lll_nomax_bjets, axis=1) == ak.argmax(lll_nomax_bjets.pt, axis=1)]
                lll_max_jetcross = ak.cartesian({'i0': lll_fwd_lightjets, 'i1': lll_max_bjets})
                lll_submax_jetcross = ak.cartesian({'i0': lll_fwd_lightjets, 'i1': lll_submax_bjets})
                lll_fwdJet_leadbJet = np.abs(lll_max_jetcross['i0'].eta - lll_max_jetcross['i1'].eta)
                lll_fwdJet_subleadbJet = np.abs(lll_submax_jetcross['i0'].eta - lll_submax_jetcross['i1'].eta)
                lll_fwdJet_closestLepton = ak.min(ak.flatten(lll_fwd_lightjets.eta) - tight_lll.eta, axis=1)
                lll_pairs = ak.combinations(tight_lll, 2, fields=['i0', 'i1'])
                lll_maxpt_indices = ak.argmax((lll_pairs.i0 + lll_pairs.i1).pt, axis=1)
                lll_maxpt_pairs = lll_pairs[ak.local_index(lll_pairs) == lll_maxpt_indices]
                lll_highpT_sslp = ak.flatten(np.abs(lll_maxpt_pairs.i0.phi - lll_maxpt_pairs.i1.phi))
                lll_jetMaxEta = ak.fill_none(ak.max(np.abs(jets_3lep.eta[(jets_3lep.btagCSVV2 <= loose_wp) & (jets_3lep.pt > 40)]), axis=1), 0)
                lll_minDelR = ak.min(lll_pairs.i0.delta_r(lll_pairs.i1), axis=1)
                lll_nJet25 = ak.num(jets_3lep[(jets_3lep.pt > 25) & (np.abs(jets_3lep.eta) < 2.4)])
                lll_nJetEta1 = ak.num(jets_3lep[(jets_3lep.btagCSVV2 <= loose_wp) & (np.abs(jets_3lep.eta) > 1)])
                lll_TotalCharge = ak.sum(tight_lll.charge, axis=1)

                output['lll_3rdLepton_pT'].fill(dataset=dataset, lll_3rdLepton_pT=lll_3rdLepton_pT)
                output['lll_fwdJet_leadbJet'].fill(dataset=dataset, lll_fwdJet_leadbJet=ak.flatten(lll_fwdJet_leadbJet))
                output['lll_fwdJet_subleadbJet'].fill(dataset=dataset, lll_fwdJet_subleadbJet=ak.flatten(lll_fwdJet_subleadbJet))
                output['lll_fwdJet_closestLepton'].fill(dataset=dataset, lll_fwdJet_closestLepton=lll_fwdJet_closestLepton)
                output['lll_highpT_sslp'].fill(dataset=dataset, lll_highpT_sslp=lll_highpT_sslp)
                output['lll_jetMaxEta'].fill(dataset=dataset, lll_jetMaxEta=lll_jetMaxEta)
                output['lll_minDelR'].fill(dataset=dataset, lll_minDelR=lll_minDelR)
                output['lll_nJet25'].fill(dataset=dataset, lll_nJet25=lll_nJet25)
                output['lll_nJetEta1'].fill(dataset=dataset, lll_nJetEta1=lll_nJetEta1)
                output['lll_TotalCharge'].fill(dataset=dataset, lll_TotalCharge=lll_TotalCharge)
        
        if (di_empty == False):
            # mu-mu channel
            if ak.size(tights_2mu, axis=0) > 0:
                mumu_subleadLepton_pT = tights_2mu.pt[:, 1]
                mumu_lightjets = jets_2mu[jets_2mu.btagCSVV2 <= loose_wp]
                mumu_bjets = jets_2mu[jets_2mu.btagCSVV2 > medium_wp]
                mumu_fwd_lightjets = mumu_lightjets[ak.local_index(mumu_lightjets, axis=1) == ak.argmax(mumu_lightjets.eta, axis=1)]
                mumu_max_bjets = mumu_bjets[ak.local_index(mumu_bjets, axis=1) == ak.argmax(mumu_bjets.pt, axis=1)]
                mumu_nomax_bjets = mumu_bjets[ak.local_index(mumu_bjets, axis=1) != ak.argmax(mumu_bjets.pt, axis=1)]
                mumu_submax_bjets = mumu_nomax_bjets[ak.local_index(mumu_nomax_bjets, axis=1) == ak.argmax(mumu_nomax_bjets.pt, axis=1)]
                mumu_max_jetcross = ak.cartesian({'i0': mumu_fwd_lightjets, 'i1': mumu_max_bjets})
                mumu_submax_jetcross = ak.cartesian({'i0': mumu_fwd_lightjets, 'i1': mumu_submax_bjets})
                mumu_fwdJet_leadbJet = np.abs(mumu_max_jetcross['i0'].eta - mumu_max_jetcross['i1'].eta)
                mumu_fwdJet_subleadbJet = np.abs(mumu_submax_jetcross['i0'].eta - mumu_submax_jetcross['i1'].eta)
                mumu_fwdJet_closestLepton = ak.min(ak.flatten(mumu_fwd_lightjets.eta) - tights_2mu.eta, axis=1)
                mumu_highpT_sslp = np.abs(tights_2mu.phi[:, 0] - tights_2mu.phi[:, 1])
                mumu_jetMaxEta = ak.fill_none(ak.max(np.abs(jets_2mu.eta[(jets_2mu.btagCSVV2 <= loose_wp) & (jets_2mu.pt > 40)]), axis=1), 0)
                mumu_minDelR = tights_2mu[:, 0].delta_r(tights_2mu[:, 1])
                mumu_nJet25 = ak.num(jets_2mu[(jets_2mu.pt > 25) & (np.abs(jets_2mu.eta) < 2.4)])
                mumu_nJetEta1 = ak.num(jets_2mu[(jets_2mu.btagCSVV2 <= loose_wp) & (np.abs(jets_2mu.eta) > 1)])
                mumu_TotalCharge = ak.sum(tights_2mu.charge, axis=1)

                output['mumu_subleadLepton_pT'].fill(dataset=dataset, mumu_subleadLepton_pT=mumu_subleadLepton_pT)
                output['mumu_fwdJet_leadbJet'].fill(dataset=dataset, mumu_fwdJet_leadbJet=ak.flatten(mumu_fwdJet_leadbJet))
                output['mumu_fwdJet_subleadbJet'].fill(dataset=dataset, mumu_fwdJet_subleadbJet=ak.flatten(mumu_fwdJet_subleadbJet))
                output['mumu_fwdJet_closestLepton'].fill(dataset=dataset, mumu_fwdJet_closestLepton=mumu_fwdJet_closestLepton)
                output['mumu_highpT_sslp'].fill(dataset=dataset, mumu_highpT_sslp=mumu_highpT_sslp)
                output['mumu_jetMaxEta'].fill(dataset=dataset, mumu_jetMaxEta=mumu_jetMaxEta)
                output['mumu_minDelR'].fill(dataset=dataset, mumu_minDelR=mumu_minDelR)
                output['mumu_nJet25'].fill(dataset=dataset, mumu_nJet25=mumu_nJet25)
                output['mumu_nJetEta1'].fill(dataset=dataset, mumu_nJetEta1=mumu_nJetEta1)
                output['mumu_TotalCharge'].fill(dataset=dataset, mumu_TotalCharge=mumu_TotalCharge)

            # e-mu channel
            if ak.size(tights_emu, axis=0) > 0:
                emu_subleadLepton_pT = tights_emu.pt[:, 1]
                emu_lightjets = jets_emu[jets_emu.btagCSVV2 <= loose_wp]
                emu_bjets = jets_emu[jets_emu.btagCSVV2 > medium_wp]
                emu_fwd_lightjets = emu_lightjets[ak.local_index(emu_lightjets, axis=1) == ak.argmax(emu_lightjets.eta, axis=1)]
                emu_max_bjets = emu_bjets[ak.local_index(emu_bjets, axis=1) == ak.argmax(emu_bjets.pt, axis=1)]
                emu_nomax_bjets = emu_bjets[ak.local_index(emu_bjets, axis=1) != ak.argmax(emu_bjets.pt, axis=1)]
                emu_submax_bjets = emu_nomax_bjets[ak.local_index(emu_nomax_bjets, axis=1) == ak.argmax(emu_nomax_bjets.pt, axis=1)]
                emu_max_jetcross = ak.cartesian({'i0': emu_fwd_lightjets, 'i1': emu_max_bjets})
                emu_submax_jetcross = ak.cartesian({'i0': emu_fwd_lightjets, 'i1': emu_submax_bjets})
                emu_fwdJet_leadbJet = np.abs(emu_max_jetcross['i0'].eta - emu_max_jetcross['i1'].eta)
                emu_fwdJet_subleadbJet = np.abs(emu_submax_jetcross['i0'].eta - emu_submax_jetcross['i1'].eta)
                emu_fwdJet_closestLepton = ak.min(ak.flatten(emu_fwd_lightjets.eta) - tights_emu.eta, axis=1)
                emu_highpT_sslp = np.abs(tights_emu.phi[:, 0] - tights_emu.phi[:, 1])
                emu_jetMaxEta = ak.fill_none(ak.max(np.abs(jets_emu.eta[(jets_emu.btagCSVV2 <= loose_wp) & (jets_emu.pt > 40)]), axis=1), 0)
                emu_minDelR = tights_emu[:, 0].delta_r(tights_emu[:, 1])
                emu_nJet25 = ak.num(jets_emu[(jets_emu.pt > 25) & (np.abs(jets_emu.eta) < 2.4)])
                emu_nJetEta1 = ak.num(jets_emu[(jets_emu.btagCSVV2 <= loose_wp) & (np.abs(jets_emu.eta) > 1)])
                emu_TotalCharge = ak.sum(tights_emu.charge, axis=1)

                output['emu_subleadLepton_pT'].fill(dataset=dataset, emu_subleadLepton_pT=emu_subleadLepton_pT)
                output['emu_fwdJet_leadbJet'].fill(dataset=dataset, emu_fwdJet_leadbJet=ak.flatten(emu_fwdJet_leadbJet))
                output['emu_fwdJet_subleadbJet'].fill(dataset=dataset, emu_fwdJet_subleadbJet=ak.flatten(emu_fwdJet_subleadbJet))
                output['emu_fwdJet_closestLepton'].fill(dataset=dataset, emu_fwdJet_closestLepton=emu_fwdJet_closestLepton)
                output['emu_highpT_sslp'].fill(dataset=dataset, emu_highpT_sslp=emu_highpT_sslp)
                output['emu_jetMaxEta'].fill(dataset=dataset, emu_jetMaxEta=emu_jetMaxEta)
                output['emu_minDelR'].fill(dataset=dataset, emu_minDelR=emu_minDelR)
                output['emu_nJet25'].fill(dataset=dataset, emu_nJet25=emu_nJet25)
                output['emu_nJetEta1'].fill(dataset=dataset, emu_nJetEta1=emu_nJetEta1)
                output['emu_TotalCharge'].fill(dataset=dataset, emu_TotalCharge=emu_TotalCharge)

            # e-e channel
            if ak.size(tights_2e, axis=0) > 0:
                ee_subleadLepton_pT = tights_2e.pt[:, 1]
                ee_lightjets = jets_2e[jets_2e.btagCSVV2 <= loose_wp]
                ee_bjets = jets_2e[jets_2e.btagCSVV2 > medium_wp]
                ee_fwd_lightjets = ee_lightjets[ak.local_index(ee_lightjets, axis=1) == ak.argmax(ee_lightjets.eta, axis=1)]
                ee_max_bjets = ee_bjets[ak.local_index(ee_bjets, axis=1) == ak.argmax(ee_bjets.pt, axis=1)]
                ee_nomax_bjets = ee_bjets[ak.local_index(ee_bjets, axis=1) != ak.argmax(ee_bjets.pt, axis=1)]
                ee_submax_bjets = ee_nomax_bjets[ak.local_index(ee_nomax_bjets, axis=1) == ak.argmax(ee_nomax_bjets.pt, axis=1)]
                ee_max_jetcross = ak.cartesian({'i0': ee_fwd_lightjets, 'i1': ee_max_bjets})
                ee_submax_jetcross = ak.cartesian({'i0': ee_fwd_lightjets, 'i1': ee_submax_bjets})
                ee_fwdJet_leadbJet = np.abs(ee_max_jetcross['i0'].eta - ee_max_jetcross['i1'].eta)
                ee_fwdJet_subleadbJet = np.abs(ee_submax_jetcross['i0'].eta - ee_submax_jetcross['i1'].eta)
                ee_fwdJet_closestLepton = ak.min(ak.flatten(ee_fwd_lightjets.eta) - tights_2e.eta, axis=1)
                ee_highpT_sslp = np.abs(tights_2e.phi[:, 0] - tights_2e.phi[:, 1])
                ee_jetMaxEta = ak.fill_none(ak.max(np.abs(jets_2e.eta[(jets_2e.btagCSVV2 <= loose_wp) & (jets_2e.pt > 40)]), axis=1), 0)
                ee_minDelR = tights_2e[:, 0].delta_r(tights_2e[:, 1])
                ee_nJet25 = ak.num(jets_2e[(jets_2e.pt > 25) & (np.abs(jets_2e.eta) < 2.4)])
                ee_nJetEta1 = ak.num(jets_2e[(jets_2e.btagCSVV2 <= loose_wp) & (np.abs(jets_2e.eta) > 1)])
                ee_TotalCharge = ak.sum(tights_2e.charge, axis=1)

                output['ee_subleadLepton_pT'].fill(dataset=dataset, ee_subleadLepton_pT=ee_subleadLepton_pT)
                output['ee_fwdJet_leadbJet'].fill(dataset=dataset, ee_fwdJet_leadbJet=ak.flatten(ee_fwdJet_leadbJet))
                output['ee_fwdJet_subleadbJet'].fill(dataset=dataset, ee_fwdJet_subleadbJet=ak.flatten(ee_fwdJet_subleadbJet))
                output['ee_fwdJet_closestLepton'].fill(dataset=dataset, ee_fwdJet_closestLepton=ee_fwdJet_closestLepton)
                output['ee_highpT_sslp'].fill(dataset=dataset, ee_highpT_sslp=ee_highpT_sslp)
                output['ee_jetMaxEta'].fill(dataset=dataset, ee_jetMaxEta=ee_jetMaxEta)
                output['ee_minDelR'].fill(dataset=dataset, ee_minDelR=ee_minDelR)
                output['ee_nJet25'].fill(dataset=dataset, ee_nJet25=ee_nJet25)
                output['ee_nJetEta1'].fill(dataset=dataset, ee_nJetEta1=ee_nJetEta1)
                output['ee_TotalCharge'].fill(dataset=dataset, ee_TotalCharge=ee_TotalCharge)

        return output

    def postprocess(self, accumulator):
        lum = 35900
        
        # Calculates the number of events in every file we process.
        eventries = {}
        sumcheck = 0
        for key in fileset:
            eventries[key] = uproot3.numentries(fileset[key], 'Events')
            sumcheck += uproot3.numentries(fileset[key], 'Events')
            
        print(eventries)
        print(sumcheck)
        
        # Cross-sections are found in the draft note. Weights follow the formula of [cross-section * integrate luminosity]/[events]
        weights = {'tHq': (lum*0.7927)/eventries['tHq'],
                      'tHW': (lum*0.1472)/eventries['tHW'],
                      'ttW': (lum*0.6105)/eventries['ttW'],
                      'ttWJets': (lum*0.2043)/eventries['ttWJets'],
                      'ttZ': (lum*0.2529)/eventries['ttZ'],
                      'ttZJets': (lum*0.5297)/eventries['ttZJets'], # cross-section is labaled as 0.5297 / 0.692 ?
                      'ttH': (lum*0.2151)/eventries['ttH'],
                      'WZ': (lum*4.4296)/eventries['WZ'],
                      'ZZ': (lum*1.256)/eventries['ZZ'],
                      'ZZZ': (lum*0.01398)/eventries['ZZZ'],
                      'WWZ': (lum*0.1651)/eventries['WWZ'],
                      'WZZ': (lum*0.05565)/eventries['WZZ'],
                      'tttt': (lum*0.009103)/eventries['tttt'],
                      'WpWp': (lum*0.03711)/eventries['WpWp'],
                   }
        scaled = hist.Cat('scaled', 'scaled')
        for key in accumulator:
            if accumulator[key] != accumulator['cutflow']:
                # Scales each dataset along the dataset axis according to weights above.
                accumulator[key].scale(weights, axis='dataset')
                # Combines keys after scaling.
                accumulator[key] = accumulator[key].group('dataset', scaled, {'tHq': ['tHq'], 
                                                                               'tHW': ['tHW'],
                                                                               'ttW': ['ttW', 'ttWJets'],
                                                                               'ttZ': ['ttZ', 'ttZJets'],
                                                                               'ttH': ['ttH'],
                                                                               'WZ': ['WZ'],
                                                                               'ZZ': ['ZZ'],
                                                                               'VVV': ['ZZZ', 'WWZ', 'WZZ'],
                                                                               'tttt': ['tttt'],
                                                                               'WpWp': ['WpWp'],
                                                                              })
        return accumulator

In [ ]:
import time

tic = time.time()
output = processor.run_uproot_job(fileset,
                                treename = 'Events',
                                processor_instance = SignalProcessor(),
                                executor = processor.futures_executor,
                                executor_args = {'schema': processor.NanoAODSchema, 'savemetrics': True}
                                )
toc = time.time()

print("Dask client:", client)
print("Total time: %.0f" % (toc - tic))
print("Events / s / thread: {:,.0f}".format(output[1]['entries'].value / output[1]['processtime'].value))
print("Events / s: {:,.0f}".format(output[1]['entries'].value / (toc - tic)))

Received SIGINT, killed pending jobs.  Running jobs will continue to completion.
Running jobs: 9


In [ ]:
colors = ['#99CC66', '#669966', '#CC6600', '#FF3300', '#FF9966', '#CC66FF', '#FFCC00', '#000000']


stack = True
clear = False
fill_opts = None
overlay = 'scaled'

In [ ]:
print('Three Lepton Channel')
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['lll_3rdLepton_pT'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['lll_fwdJet_leadbJet'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['lll_fwdJet_subleadbJet'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['lll_fwdJet_closestLepton'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['lll_highpT_sslp'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['lll_jetMaxEta'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['lll_minDelR'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['lll_NJetMED'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['lll_NJetNONMED'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['lll_TotalCharge'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()

In [ ]:
print('mu-mu Channel')
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['mumu_subleadLepton_pT'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['mumu_fwdJet_leadbJet'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['mumu_fwdJet_subleadbJet'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['mumu_fwdJet_closestLepton'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['mumu_highpT_sslp'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['mumu_jetMaxEta'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['mumu_minDelR'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['mumu_NJetMED'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['mumu_NJetNONMED'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['mumu_TotalCharge'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()

In [ ]:
print('e-mu Channel')
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['emu_subleadLepton_pT'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['emu_fwdJet_leadbJet'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['emu_fwdJet_subleadbJet'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['emu_fwdJet_closestLepton'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['emu_highpT_sslp'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['emu_jetMaxEta'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['emu_minDelR'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['emu_NJetMED'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['emu_NJetNONMED'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['emu_TotalCharge'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()

In [ ]:
print('e-e Channel')
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['ee_subleadLepton_pT'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['ee_fwdJet_leadbJet'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['ee_fwdJet_subleadbJet'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['ee_fwdJet_closestLepton'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['ee_highpT_sslp'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['ee_jetMaxEta'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['ee_minDelR'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['ee_NJetMED'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['ee_NJetNONMED'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()
fig, ax = plt.subplots(1)
ax.set_prop_cycle(cycler(color=colors))
hist.plot1d(output['ee_TotalCharge'], overlay=overlay, stack=stack, ax=ax, clear=clear, fill_opts=fill_opts)
ax.set_ylim(0, None)
ax.legend()

In [ ]:
for key, value in output['cutflow'].items():
    print(key, value)